In [ ]:
# https://github.com/PraveshKoirala/nepali_morpheme_splitter/blob/master/separate_morphemes.ipynb 
# https://medium.com/@shivagaire/%E0%A4%9B%E0%A4%A8%E0%A5%8D%E0%A4%A6%E0%A4%AE%E0%A4%BE-%E0%A4%95%E0%A4%B5%E0%A4%BF%E0%A4%A4%E0%A4%BE-%E0%A4%95%E0%A4%B8%E0%A4%B0%E0%A5%80-%E0%A4%B2%E0%A5%87%E0%A4%96%E0%A5%8D%E0%A4%A8%E0%A5%87-7782c0a01967 

In [6]:
import re
import pandas as pd
from IPython.display import display

# --- परिभाषाहरू (Definitions) ---
vowel_pattern = r"[\u0905-\u0914]"  # Independent vowels: अ to औ
matra_modifier_pattern = r"[\u093A-\u094C\u0901-\u0903]" # Matras and modifiers
consonant_pattern = r"[क-हज्ञक्ष]" # Consonants including conjuncts
halanta = "्"

# --- सहयोगी Function हरू (Helper Functions) ---
def is_vowel(char):
    return bool(re.match(vowel_pattern, char))

def is_matra_modifier(char):
    return bool(re.match(matra_modifier_pattern, char))

def is_consonant(char):
    return bool(re.match(consonant_pattern, char))

def is_halant(char):
    return char == halanta

def split_syllables(word, split_halanta=False):
    """
    यो function ले नेपाली शब्दलाई त्यसको अक्षर (syllables) मा टुक्राउँछ।

    Args:
        word (str): विभाजन गर्नुपर्ने नेपाली शब्द।
        split_halanta (bool): 
            - False (Default): भाषाको नियमअनुसार सही अक्षरमा विभाजन गर्छ (e.g., 'स्नु' एउटै अक्षर)।
            - True: हल्न्त लागेको हरेक आधा व्यञ्जनलाई छुट्टै टुक्रा मान्छ (e.g., 'स्' र 'नु' छुट्टाछुट्टै)।
    """
    if not re.fullmatch(r"[\u0900-\u097F]+", word):
        print(f"Error: '{word}' मा अमान्य अक्षरहरू छन्।")
        return []

    syllables = []
    current_syllable = ""
    i = 0
    word_len = len(word)

    while i < word_len:
        char = word[i]

        if is_consonant(char):
            current_syllable += char
            
            next_i = i + 1
            if next_i < word_len:
                next_char = word[next_i]
                
                # --- परिमार्जित तर्क (Modified Logic) ---
                if is_halant(next_char):
                    current_syllable += next_char
                    i += 1
                    
                    if split_halanta:
                        # यदि split_halanta=True छ भने, आधा अक्षरलाई छुट्टै टुक्रा बनाउने
                        syllables.append(current_syllable)
                        current_syllable = ""

                elif is_matra_modifier(next_char):
                    current_syllable += next_char
                    i += 1
                    while i + 1 < word_len and is_matra_modifier(word[i + 1]):
                        current_syllable += word[i + 1]
                        i += 1
                    syllables.append(current_syllable)
                    current_syllable = ""
                else:
                    syllables.append(current_syllable)
                    current_syllable = ""
            else:
                syllables.append(current_syllable)
                current_syllable = ""

        elif is_vowel(char):
            if current_syllable:
                syllables.append(current_syllable)
            
            current_syllable = char
            while i + 1 < word_len and is_matra_modifier(word[i + 1]):
                current_syllable += word[i + 1]
                i += 1
            
            syllables.append(current_syllable)
            current_syllable = ""
        
        i += 1

    if current_syllable and syllables and not split_halanta:
        syllables[-1] += current_syllable
    elif current_syllable:
        syllables.append(current_syllable)


    return syllables

# --- Jupyter Notebook मा चलाउने उदाहरण ---

def display_syllables(word):
    """परिणामलाई दुवै तरिकाले देखाउने helper function"""
    print(f"'{word}' शब्दको अक्षर विभाजन:")
    
    # अवस्था १: Default (split_halanta=False) - सही अक्षर विभाजन
    print("\n१. पूर्वनिर्धारित (Default) विभाजन (split_halanta=False):")
    syllables_default = split_syllables(word) # or split_syllables(word, split_halanta=False)
    if syllables_default:
        display(pd.DataFrame(syllables_default, columns=["अक्षर (Syllable)"]))

    # अवस्था २: हल्न्तमा विभाजन (split_halanta=True)
    print("\n२. हल्न्तमा विभाजन (split_halanta=True):")
    syllables_split = split_syllables(word, split_halanta=True)
    if syllables_split:
        display(pd.DataFrame(syllables_split, columns=["टुक्रा (Segment)"]))
        
    print("-" * 40)


# उदाहरण १: 'हाँस्नु'
display_syllables("हाँस्नु")

# उदाहरण २: 'विद्यार्थी'
display_syllables("विद्यार्थी")

# उदाहरण ३: 'संविधान'
display_syllables("श्रीमान")

'हाँस्नु' शब्दको अक्षर विभाजन:

१. पूर्वनिर्धारित (Default) विभाजन (split_halanta=False):


,अक्षर (Syllable)
0,हाँ
1,स्नु



२. हल्न्तमा विभाजन (split_halanta=True):


,टुक्रा (Segment)
0,हाँ
1,स्
2,नु


----------------------------------------
'विद्यार्थी' शब्दको अक्षर विभाजन:

१. पूर्वनिर्धारित (Default) विभाजन (split_halanta=False):


...

----------------------------------------
'श्रीमान' शब्दको अक्षर विभाजन:

१. पूर्वनिर्धारित (Default) विभाजन (split_halanta=False):


,अक्षर (Syllable)
0,श्री
1,मा
2,न



२. हल्न्तमा विभाजन (split_halanta=True):


,टुक्रा (Segment)
0,श्
1,री
2,मा
3,न


----------------------------------------


In [20]:
display_syllables("छन्")

'छन्' शब्दको अक्षर विभाजन:

१. पूर्वनिर्धारित (Default) विभाजन (split_halanta=False):


,अक्षर (Syllable)
0,छन्



२. हल्न्तमा विभाजन (split_halanta=True):


,टुक्रा (Segment)
0,छ
1,न्


----------------------------------------


In [13]:
# All known Chhanda patterns for reference
CHHANDA_PATTERNS = {
    "शार्दूलविक्रीडित": ["SSS IIS ISI IIS SSI SSI S", 19],
    "भुजङ्गप्रयात": ["ISS ISS ISS ISS", 12],
    "तोडक": ["IIS IIS IIS IIS", 12],
    "शिखरिणी": ["ISS SSS III IIS SII I S", 17],
    "मन्दाक्रान्ता": ["SSS SII III SSI SSI S S", 17],
    "वंशस्थ": ["ISI SSI ISI SIS", 12],
    "वसन्ततिलका": ["SSI SII ISI ISI SS", 14],
    "मालिनी": ["III III SSS ISS ISS", 15],
    "स्रग्धरा": ["SSS SIS SII III ISS ISS ISS", 21],
    "जलधरमाला": ["SSSS IIII SSSS", 12],
    "कुसुमविचित्रा": ["III ISS III ISS", 12],
    "अलोला": ["SSS IIS SSS SII S S", 14],
    "विद्युन्माला": ["SSS SSS S S", 8],
    "स्रग्विणी": ["SIS SIS SIS SIS", 12],
    "विद्युल्लेखा": ["S S S S S S", 6],
    "प्रमाणिका": ["ISI SIS I S", 8],
    "चम्पकमाला": ["SII SSS IIS S", 10],
    "इन्द्रवज्रा": ["SSI SSI ISI S S", 11],
    "श्री": ["S", 1],
    "स्वागता": ["SIS III SII S S", 11],
    "रथोद्धता": ["SIS III SIS I S", 11],
    "इन्दिरा": ["III SIS SIS I S", 11],
    "रूपामाली": ["SSS SSS SSS", 9],
    "उपेन्द्रवज्रा": ["ISI SSI ISI S S", 11],
    "शालिनी": ["SSS SSI SSI S S", 11],
    "मालती": ["III ISI ISI SIS", 12],
    "द्रुतविलम्बित": ["III SII SII SIS", 12],
    "मणिमाला": ["SII SSS IIS", 8],
    "पञ्चचामर": ["ISI SIS ISI SIS ISI S", 16],
    "अपराजिता": ["III III SIS IIS I S", 14],
    "प्रीतिमाला": ["SSS ISI SSI ISI SII S", 16],
    "मदिरा": ["SII SII SII SII SII SII SII S", 22],
    "मेघविस्फूर्जिता": ["ISS SSS III IIS SIS SIS S S", 19],
    "प्रहर्षिणी": ["SSS III ISI SIS S", 13],
    "प्रमिताक्षरा": ["IIS ISI IIS IIS", 12],
    "पृथ्वी": ["ISI IIS ISI IIS ISS I S", 17],
    "दुर्मिला": ["IIS IIS IIS IIS IIS IIS IIS IIS", 24],
    "विधाता": ["ISS SIS SSI SSS ISS S", 16],
    "मनिका": ["IIS ISS ISS SIS S", 13],
    "हरिणी": ["III IIS SSS SIS IIS I S", 17],
    "दिक्पाल": ["SSI SIS SSS ISI S S", 14],
    "चित्रपदा": ["SII SII S S", 8],
    "माद्री": ["ISI SSI ISI S", 10],
    "सुप्रतिष्ठा": ["SII S S", 5],
    "तूर्णक": ["SIS ISI SIS ISI SIS", 15],
    "सत्यवती": ["SII SSI SIS S", 11],
    "चित्रिका": ["SIS ISI SIS ISI SIS", 15],
    "इन्द्रवंशा": ["SSI SSI ISI SIS", 12],
    "रुक्मवती": ["SII SSS IIS S", 10],
    "शङ्खनारी": ["ISS ISS", 6],
    "अचलधृति": ["IIIIIIIIIIIIIIII", 16],
    "अनुकुला": ["SII SSI III S S", 11]
}

In [17]:
import re
import pandas as pd
from collections import Counter



# --- Core Logic: Syllabification and Mātrā Parsing ---

# User's trusted syllabification function
vowel_pattern = r"[\u0905-\u0914]"
matra_modifier_pattern = r"[\u093A-\u094C\u0901-\u0903]"
consonant_pattern = r"[क-हज्ञक्ष]"
halanta = "्"

def is_vowel(char): return bool(re.match(vowel_pattern, char))
def is_matra_modifier(char): return bool(re.match(matra_modifier_pattern, char))
def is_consonant(char): return bool(re.match(consonant_pattern, char))
def is_halant(char): return char == halanta

def split_syllables(word, split_halanta=False):
    word = "".join(word.split())
    if not word: return []
    syllables = []
    current_syllable = ""
    i = 0
    word_len = len(word)
    while i < word_len:
        char = word[i]
        if is_consonant(char):
            current_syllable += char
            next_i = i + 1
            if next_i < word_len:
                next_char = word[next_i]
                if is_halant(next_char):
                    current_syllable += next_char; i += 1
                    if split_halanta: syllables.append(current_syllable); current_syllable = ""
                elif is_matra_modifier(next_char):
                    current_syllable += next_char; i += 1
                    while i + 1 < word_len and is_matra_modifier(word[i + 1]):
                        current_syllable += word[i + 1]; i += 1
                    syllables.append(current_syllable); current_syllable = ""
                else:
                    syllables.append(current_syllable); current_syllable = ""
            else:
                syllables.append(current_syllable); current_syllable = ""
        elif is_vowel(char):
            if current_syllable: syllables.append(current_syllable)
            current_syllable = char
            while i + 1 < word_len and is_matra_modifier(word[i + 1]):
                current_syllable += word[i + 1]; i += 1
            syllables.append(current_syllable); current_syllable = ""
        i += 1
    if current_syllable and syllables and not split_halanta: syllables[-1] += current_syllable
    elif current_syllable: syllables.append(current_syllable)
    return syllables



def parse_matra_pattern(line):
    syllables = split_syllables(line)
    if not syllables: return "", []
    
    matras = []
    num_syllables = len(syllables)
    long_matra_chars = "ाीूेैोौ" + "आईऊएऐओऔ"; modifier_chars = "ंः"
    
    for i, syl in enumerate(syllables):
        is_guru = False
        if i == num_syllables - 1: is_guru = True
        elif any(c in long_matra_chars for c in syl): is_guru = True
        elif any(c in modifier_chars for c in syl): is_guru = True
        elif syl.endswith(halanta): is_guru = True
        elif (i + 1 < num_syllables) and (halanta in syllables[i+1]): is_guru = True
        matras.append('S' if is_guru else 'I')
    print("".join(matras), syllables)
        
    return "".join(matras), syllables

# --- Fuzzy Logic: Levenshtein Distance ---
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2): return levenshtein_distance(s2, s1)
    if len(s2) == 0: return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

# --- Main Analysis Engine ---
def analyze_poem(poem_text):
    lines = [line.strip() for line in poem_text.strip().split('\n') if line.strip()]
    if not lines:
        print("Poem is empty.")
        return

    # 1. Suggest possibilities based on first line's syllable count
    first_line_pattern, first_line_syllables = parse_matra_pattern(lines[0])
    syllable_count = len(first_line_syllables)
    possible_chhandas = [name for name, (_, length) in CHHANDA_PATTERNS.items() if length == syllable_count]

    print(f"विश्लेषण: '{lines[0]}...'")
    print("-" * 30)
    print(f" syllable count ({syllable_count}) Chhandas: {', '.join(possible_chhandas) or 'None'}")
    
    line_results = []
    overall_matches = []

    # 2. Analyze each line for exact and fuzzy matches
    for i, line in enumerate(lines):
        line_pattern, syllables = parse_matra_pattern(line)
        if not line_pattern: continue

        exact_match = "None"
        for name, (pattern, length) in CHHANDA_PATTERNS.items():
            if len(syllables) == length and pattern.replace(" ", "") == line_pattern:
                exact_match = name
                overall_matches.append(name)
                break
        
        # Fuzzy match if no exact match found
        closest_match = "None"
        if exact_match == "None":
            min_dist = float('inf')
            for name, (pattern, length) in CHHANDA_PATTERNS.items():
                if len(syllables) == length:
                    dist = levenshtein_distance(line_pattern, pattern.replace(" ", ""))
                    if dist < min_dist:
                        min_dist = dist
                        closest_match = f"{name} (diff: {dist})"
            overall_matches.append(closest_match.split(" (")[0])


        line_results.append(f"Line {i+1}: Pattern={line_pattern}, Exact Match='{exact_match}', Closest Match='{closest_match}'")

    print("\nDetailed Line-by-Line Analysis:")
    for res in line_results:
        print(res)

    # 3. Conclude the most suitable Chhanda
    if overall_matches:
        most_common = Counter(overall_matches).most_common(1)[0][0]
        print(f"\nMost Suitable Chhanda: {most_common}")
    else:
        print("\nCould not determine a suitable Chhanda.")
    print("=" * 40 + "\n")


In [18]:
CHHANDA_EXAMPLES = {
  "शार्दूलविक्रीडित": {
    "chhanda": "शार्दूलविक्रीडित",
    "lines": "कन्यादान भयाे थिए दुईजना भाेका भगत् धर्मका\nपानीदार विहा गरेर अँजुली जाेडी दिए कर्मका"
  },
  "भुजङ्गप्रयात": {
    "chhanda": "भुजङ्गप्रयात",
    "lines": "दिई दर्द झन् सर्प डस्दैछ ऐले\nछिमेकी बिना शर्म पस्दैछ ऐले"
  },
  "अनुष्टुप": {
    "chhanda": "अनुष्टुप",
    "lines": "देश छैन पिता छैन माता छैन न मित्र छन्\nन बधु छन् न स्वास्नी छन् न कतै घरवार छन्"
  },
  "मन्दाक्रान्ता": {
    "chhanda": "मन्दाक्रान्ता",
    "lines": "छाना लाए घर हुन गयाे जिन्दगी छैन भित्र\nमुर्दा मुर्दा दुई दिल विहा गर्दछन् याे विचित्र"
  },
  "जलधरमाला": {
    "chhanda": "जलधरमाला",
    "lines": "बाबा आमा हितकर बोली भन्छन्\nसाना नानी अहित भनेझैं ठान्छन्"
  },
  "कुसुमविचित्रा": {
    "chhanda": "कुसुमविचित्रा",
    "lines": "बुझ न मधेसी मुलुक नभाँड\nरिपुसँग मिल्दै ह्रदय नफाँड"
  },
  "शिखरिणी": {
    "chhanda": "शिखरिणी",
    "lines": "छ धर्तीमा शोभा अनुपम कतै प्रस्ट छ तर\nकतै लुक्छन् भित्रै प्रकटित नभै चित्र लहर ।"
  },
  "अलोला": {
    "chhanda": "अलोला",
    "lines": "तारा सुन्दर चम्कून् धर्ती उज्ज्वल होऊन् ।\nनेपाली मन मैलो वर्षाले सरि धोऊन् ।।"
  },
  "विद्युन्माला": {
    "chhanda": "विद्युन्माला",
    "lines": "छारो हानी तर्साएर\nकालो छाया बर्साएर"
  },
  "स्रग्विणी": {
    "chhanda": "स्रग्विणी",
    "lines": "दौडिए जिन्दगी साँझमा रातमा\nनित्यको खोजमा दालमा भातमा।"
  },
  "उपजाति": {
    "chhanda": "उपजाति",
    "lines": "टुटेर नै बादल झार्छ पानी\nफुटेर नै पत्थर बन्छ खानी"
  },
  "विद्युल्लेखा": {
    "chhanda": "विद्युल्लेखा",
    "lines": "छैनन् मान्छेवादी\nकोही माओवादी"
  },
  "प्रमाणिका": {
    "chhanda": "प्रमाणिका",
    "lines": "म माझ प्रेमको चरो\nकराउँदैछ सुस्तरी"
  },
  "चम्पकमाला": {
    "chhanda": "चम्पकमाला",
    "lines": "शङ्कर बाबा शङ्कर बाबा\nदुःख निकै भो शङ्कर बाबा"
  },
  "इन्द्रवज्रा": {
    "chhanda": "इन्द्रवज्रा",
    "lines": "भाषा टिके गौरव टिक्न सक्छ\nपुस्ता नयाँले पनि सिक्न सक्छ ।।"
  },
  "श्री": {
    "chhanda": "श्री",
    "lines": "आ: यो\nके हो ?"
  },
  "स्वागता": {
    "chhanda": "स्वागता",
    "lines": "अङ्ग-भङ्ग हुन गो गरिखाने\nछैन चाहत उसै मरिजाने ।"
  },
  "रथोद्धता": {
    "chhanda": "रथोद्धता",
    "lines": "भो भयो अब यता नफर्कनू\nदुख्छ घाउ जल भै नदर्कनू"
  },
  "वियोगिनी": {
    "chhanda": "वियोगिनी",
    "lines": "तनमा कुन तत्व पो पस्यो ?\nमनमा त्यो किन गै कता बस्यो?"
  },
  "वसन्त मालिका": {
    "chhanda": "वसन्त मालिका",
    "lines": "शवको कुन हुन्छ कामना नै ?\nशवको के छ र भित्र भावना नै ?"
  },
  "इन्दिरा": {
    "chhanda": "इन्दिरा",
    "lines": "अब त माहुरी भुन्भुनाउँदै\nगति बढाउँदै गुन्गुनाउँदै"
  },
  "रूपामाली": {
    "chhanda": "रूपामाली",
    "lines": "हामी माली बालीनालीका\nहामी फाली मेची कालीका"
  },
  "उपेन्द्रवज्रा": {
    "chhanda": "उपेन्द्रवज्रा",
    "lines": "उडेर आकास कहाँ चुमिन्छ\nसुतेर घैटो कसरी भरिन्छ"
  },
  "शालिनी": {
    "chhanda": "शालिनी",
    "lines": "लामो यात्रा पार गर्दै म आए\nसामुन्नेमा भैदिदा लौ रमाए"
  },
  "मालती": {
    "chhanda": "मालती",
    "lines": "नयन कटाक्ष छुरा भनूँ कि के !\nछमछम बज्छ चुरा भनूँ कि के !"
  },
  "द्रुतविलम्वित": {
    "chhanda": "द्रुतविलम्वित",
    "lines": "नयनभित्र नदी छ समुद्र छ।\nतर मुहान उही छ-वि-चित्र छ।।"
  },
  "मणिमाला": {
    "chhanda": "मणिमाला",
    "lines": "खोल्दछ पर्दा त्यो नयमा\nबल्छ सधैं नै जागरमा"
  },
  "पञ्चचामर": {
    "chhanda": "पञ्चचामर",
    "lines": "बसेर देशको सधैँ गरेर हीत खातिर\nमरेर लानु के छ यो अजम्बरी त को छ र"
  },
  "अपराजिता": {
    "chhanda": "अपराजिता",
    "lines": "छल र वल गरी थुनीकन बाँधियो\nडङ कि डङ पिटी अचेत बनाइयो"
  },
  "वसन्ततिलका": {
    "chhanda": "वसन्ततिलका",
    "lines": "रात्री गयो अब प्रभात भएर आयो\nछायो सबैतिर प्रभा सविता उदायो।"
  },
  "प्रीतिमाला": {
    "chhanda": "प्रीतिमाला",
    "lines": "छाती बेस्सरी जलाई,जल नेत्रको सलाई\nफ्याक्दै पासमा नआई,घरबाट दृष्टि लाई"
  },
  "मदिरा": {
    "chhanda": "मदिरा",
    "lines": "ताल बन्यो मदिरा र कुवा पनि भो मदिरा भवसागर भो\nसृष्टि जलामय भो मदिरा तब यो दुनियाँ पनि मातिदियो"
  },
  "मेघविस्फूर्जिता": {
    "chhanda": "मेघविस्फूर्जिता",
    "lines": "लुलाले सिर्जेका अनुपम मिठा, लेख \"हावा\" हुँदै छन्\nबडाले खोकेका, अति विरसिला, शब्द \"वा वा\" हुँदैछन्"
  },
  "वंशस्थ": {
    "chhanda": "वंशस्थ",
    "lines": "पियारको वन्धन पो खडाभयो\nयहाँ तगारो मनको कडा भयो"
  },
  "प्रहर्षिणी": {
    "chhanda": "प्रहर्षिणी",
    "lines": "बच्चाले कतिथरिका मजा लिँदैछन्\nमाटोमा लुटुपुटिँदै रमाउँदैछन् ।।"
  },
  "मात्रिक": {
    "chhanda": "मात्रिक",
    "lines": "म पनि रिले अनसनमा छु\nखाएपछि हाजिर हुन्छु"
  },
  "अार्या": {
    "chhanda": "अार्या",
    "lines": "धन्य छ नारी जीवन, कसले बुझ्न सक्ला र तिम्रो मन\nभित्र वेदना दन्दन , तर पनि बाहिर मुस्कुराउँदै छन् ।।"
  },
  "प्रमिताक्षरा": {
    "chhanda": "प्रमिताक्षरा",
    "lines": "तर यार प्यार नगरी नहुने\nउनको समीप नसरी नहुने"
  },
  "तोडक": {
    "chhanda": "तोडक",
    "lines": "अबता अति भाे,अति नै खति भो।\nसबको मति लौ अति भ्रष्टभयाे।।"
  },
  "पृथ्वी": {
    "chhanda": "पृथ्वी",
    "lines": "अचम्म अति लाग्छ है किन शरीरका अङ्गमा\nविभेद नहुने कठै अधर वा कनिष्ठाङ्गमा ।"
  },
  "दुर्मिला": {
    "chhanda": "दुर्मिला",
    "lines": "जब यो युग नै अलमल्ल भयो, तब यो जग नै खलबल्ल भयो\nरस चुस्छ सधैं भमरो र जुको, अनि फूल सुकी असरल्ल भयो"
  },
  "स्रग्धरा": {
    "chhanda": "स्रग्धरा",
    "lines": "पाखामा खेतमा यी लहलह गहुँका पुष्ट बाली झुलार्इ\nमान्छेको चेतलार्इ प्रकृतिलयसँगै दूर तान्दै डुलार्इ ।"
  },
  "विधाता": {
    "chhanda": "विधाता",
    "lines": "उनी रानी म ता राजा बनेको त्यो कुरा आयो\nसगै बाचौ मरौ हामी भनेको त्यो कुरा आयो ।"
  },
  "मनिका": {
    "chhanda": "मनिका",
    "lines": "गणतन्त्र आयो अरू, के पो पुगेन?\nर शहीद,का रक्तका, टाटा सुकेन।।"
  },
  "हरिणी": {
    "chhanda": "हरिणी",
    "lines": "कुसुमदल ! फक्रेको उद्यानमा किन हो भन ?\nभ्रमरकुल झुल्दै छन् माधुर्यको कविता लिन"
  },
  "दिक्पाल": {
    "chhanda": "दिक्पाल",
    "lines": "थामेर हात तिम्रो,बाँचे म आज सम्म\nपाएर साथ तिम्रो,बाँचे म आज सम्म"
  },
  "चित्रपदा": {
    "chhanda": "चित्रपदा",
    "lines": "जान जतातिर जाऊ, यै घरमा मन लाऊ\nदेश रुने नगराऊ, जो सकिने श्रम लाऊ ।।"
  },
  "पुष्पिताग्रा": {
    "chhanda": "पुष्पिताग्रा",
    "lines": "जनन हुन समेत बाबु खोज्ने\nजनन भए उसकै छ वंश बढ्ने"
  },
  "माद्री": {
    "chhanda": "माद्री",
    "lines": "म वेदनामा कसरी बसौँ\nप्रताडनामा कसरी बसौँ"
  },
  "सुप्रतिष्ठा": {
    "chhanda": "सुप्रतिष्ठा",
    "lines": "हेर बिराला\nधूर्त छ चाला"
  },
  "तूर्णक": {
    "chhanda": "तूर्णक",
    "lines": "होस राख, दोष छोड, कर्म गर्न तम्सिद्यौ\nनिष्कलङ्क कर्मबाट बन्छ देश संझिद्यौ।"
  },
  "सत्यवती": {
    "chhanda": "सत्यवती",
    "lines": "आखिर छोटो छ जिन्दगानी\nबन्नु छ हामी सबै खरानी"
  },
  "चित्रा": {
    "chhanda": "चित्रा",
    "lines": "द्यौरालीमा पाती मेराे नाम अर्काे परेछ\nसानो छाया प्याराे लाग्थ्यो घाम चर्काे परेछ"
  },
  "चित्रिका": {
    "chhanda": "चित्रिका",
    "lines": "जाग जाग हे युवा उदार शान्त चित्तले\nहाँक्नुपर्छ देश यो सुरम्य नीति वित्तले ।"
  },
  "इन्द्रवंशा": {
    "chhanda": "इन्द्रवंशा",
    "lines": "को हुन् कुदेका किन जालझेलमा\nको हुन् डुबेका किन रक्तभेलमा"
  },
  "रुक्मवती": {
    "chhanda": "रुक्मवती",
    "lines": "भाव विनाको बन्दछ नारा।\nजून फुलाऊ व्योम सितारा।।"
  },
  "चन्द्रलेखा": {
    "chhanda": "चन्द्रलेखा",
    "lines": "आजै एकादशी माछा ता नखाऊँ तुलासी।\nअर्पौं श्री विष्णुमा माला तोरणै फूल गाँसी ।"
  },
  "शङ्खनारी": {
    "chhanda": "शङ्खनारी",
    "lines": "कथा जिन्दगीको\nथिएॅ नेत्रनानी"
  },
  "अचलधृति": {
    "chhanda": "अचलधृति",
    "lines": "अविरल चरण-रज अभिलषित म छु\nद्रवित छ हृदय तर अविचलित म छु"
  },
  "अनुकुला": {
    "chhanda": "अनुकुला",
    "lines": "हेर यहाँ छन् अतिशय पीडा\nखप्दछु जम्मै हरदम हाँसी I"
  },
  "मात्रिक (चौपाइ)": {
    "chhanda": "मात्रिक (चौपाइ)",
    "lines": "सूर्य किरणले तिमिर हरायो\nदुनियाँ उज्ज्वल मधुर गरायो!!"
  },
  "मालिनी": {
    "chhanda": "मालिनी",
    "lines": "कति छ कति गुनासो देश नै भो विरानो\nघर - शहर सबैको आँगनी भो अँध्यारो"
  }
}

In [19]:


# --- Comprehensive Testing ---
for title, data in CHHANDA_EXAMPLES.items():
    print(f"Testing Poem: {title} (Expected Chhanda: {data['chhanda']})")
    analyze_poem(data['lines'])


Testing Poem: शार्दूलविक्रीडित (Expected Chhanda: शार्दूलविक्रीडित)
SSSIISISISISSSISSIS ['क', 'न्या', 'दा', 'न', 'भ', 'याे', 'थि', 'ए', 'दु', 'ई', 'ज', 'ना', 'भाे', 'का', 'भ', 'ग', 'त्ध', 'र्म', 'का']
विश्लेषण: 'कन्यादान भयाे थिए दुईजना भाेका भगत् धर्मका...'
------------------------------
 syllable count (19) Chhandas: शार्दूलविक्रीडित, मेघविस्फूर्जिता
SSSIISISISISSSISSIS ['क', 'न्या', 'दा', 'न', 'भ', 'याे', 'थि', 'ए', 'दु', 'ई', 'ज', 'ना', 'भाे', 'का', 'भ', 'ग', 'त्ध', 'र्म', 'का']
SSSIISISIIISSSISSIS ['पा', 'नी', 'दा', 'र', 'वि', 'हा', 'ग', 'रे', 'र', 'अँ', 'जु', 'ली', 'जाे', 'डी', 'दि', 'ए', 'क', 'र्म', 'का']

Detailed Line-by-Line Analysis:
Line 1: Pattern=SSSIISISISISSSISSIS, Exact Match='None', Closest Match='शार्दूलविक्रीडित (diff: 1)'
Line 2: Pattern=SSSIISISIIISSSISSIS, Exact Match='शार्दूलविक्रीडित', Closest Match='None'

Most Suitable Chhanda: शार्दूलविक्रीडित

Testing Poem: भुजङ्गप्रयात (Expected Chhanda: भुजङ्गप्रयात)
ISSISSISSISS ['दि', 'ई', 'द', 'र्द', 'झ', 'न्स', 'र्प',